# Data Pipeline

### Our pipeline process:
#### Extract Features:
- `Airport_Pair` which will be used for `Delay_Trend_Past_Week` to see if there was a Delay trend in the previous week
- `Same_Day_Tail_Reuse`which will be using **ONLY** the date from `dep_datetime`, as well as using `Tail_Number`
- `Previous_Flight_Delay` which will be using information from `Tail_Number`, `dep_date` which will derive from `dep_datetime`, and `CRSDepTime`
- `Turnaround_Time` will also be extracted from `CRSDepTime` subtracted by `Previous_Arrival_Time`(Which is derived from `Tail_Number`, `dep_date`, and `CRSArrTime`)
- `Slack_Time` that derives from `CRSArrTime` subtracted by `CRSElaspedTime`

#### Outlier
- `Aircraft_Age` outlier must be removed 

#### Transformations: 
- `log_distance` using the log transformation 

#### Drop 
- is_Holidays

In [56]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import time


from sklearn import set_config
set_config(transform_output = "pandas")

In [57]:
def stratified_random_split(df: pd.DataFrame, target_column: str, test_size: float = 0.1, random_state: int = 42):
    """
    Performs a stratified random train-test split to ensure all classes in 
    'Flight_Status' are proportionally represented in both sets.

    Parameters:
    df (pd.DataFrame): The dataset containing the target variable.
    target_column (str): The column representing the classification target.
    test_size (float): The proportion of data to be used as test data.
    random_state (int): Random seed for reproducibility.

    Returns:
    tuple: (train_df, test_df) DataFrames.
    """
    train_df, test_df = train_test_split(
        df, test_size=test_size, stratify=df[target_column], random_state=random_state
    )

    print(f"Train size: {len(train_df)} samples")
    print(f"Test size: {len(test_df)} samples")

    return train_df, test_df

In [58]:
# Setting up the DataFrame for the flight data
flight_data = pd.read_parquet("data/WEATHER61.parquet")

# Stratify the data
train_data, test_data= stratified_random_split(flight_data, target_column="Flight_Status")

Train size: 13184010 samples
Test size: 1464890 samples


In [60]:
# downsampling the data 
train_data = train_data.sample(n=1000000, random_state=42)

# Splitting the data into X and y
X = train_data.drop(columns=["Flight_Status"])
y = train_data["Flight_Status"]

In [48]:
X

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginCityMarketID,OriginStateFips,DestAirportID,DestCityMarketID,DestStateFips,CRSDepTime,CRSArrTime,CRSElapsedTime,Distance,Aircraft_Model,Aircraft_Seats,Aircraft_Engines,Aircraft_EngineType,Is_Freighter,Aircraft_Age,Is_Holiday_Week,Holiday,dep_datetime,temperature_2m,relative_humidity_2m,precipitation,showers,snowfall,snow_depth,soil_moisture_0_to_1cm,et0_fao_evapotranspiration,evapotranspiration,cloud_cover_high,cloud_cover_low,surface_pressure,weather_code,pressure_msl,wind_direction_10m,wind_gusts_10m,dest_dep_datetime,temperature_2m_dest_dep_time,relative_humidity_2m_dest_dep_time,precipitation_dest_dep_time,showers_dest_dep_time,snowfall_dest_dep_time,snow_depth_dest_dep_time,soil_moisture_0_to_1cm_dest_dep_time,et0_fao_evapotranspiration_dest_dep_time,evapotranspiration_dest_dep_time,cloud_cover_high_dest_dep_time,cloud_cover_low_dest_dep_time,surface_pressure_dest_dep_time,weather_code_dest_dep_time,pressure_msl_dest_dep_time,wind_direction_10m_dest_dep_time,wind_gusts_10m_dest_dep_time
5556962,2023,4,10,9,1,19790,N336NW,2101.0,13487,31650,27,11637,31637,38,2255,2356,61.0,223.0,A320,NaN,2.0,Jet,False,32.6,1,Columbus Day,2023-10-09 22:55:00,6.893500,57.0,0.0,0.0,0.0,0.0,0.089,0.006185,0.0,0.0,0.0,980.69684,0.0,1011.1,291.801480,9.000000,2023-10-09 22:55:00,5.256500,70.0,0.0,0.0,0.0,0.0,0.322,0.006884,-0.0,0.0,0.0,979.65765,0.0,1012.8,333.43503,15.480000
14416945,2024,4,12,20,5,19805,N156AN,2214.0,11298,30194,48,14262,34262,6,1230,1347,197.0,1126.0,A321,181.0,2.0,Jet,False,9.1,1,Christmas Day,2024-12-20 12:30:00,12.976501,37.0,0.0,0.0,0.0,0.0,0.224,0.076416,0.0,1.0,0.0,999.01373,0.0,1021.4,206.564990,15.840000,2024-12-20 12:30:00,23.974000,10.0,0.0,0.0,0.0,0.0,0.031,0.191728,0.0,55.0,0.0,1001.62115,2.0,1016.8,150.25520,3.960000
433147,2023,1,1,24,2,19805,N129AA,696.0,11298,30194,48,12892,32575,6,1805,1929,204.0,1235.0,A321,181.0,2.0,Jet,False,10.2,0,None,2023-01-24 18:05:00,4.526500,88.0,0.0,0.0,0.0,0.0,0.461,0.030539,0.0,0.0,100.0,987.39460,3.0,1010.2,39.289394,30.599998,2023-01-24 18:05:00,16.711000,45.0,0.0,0.0,0.0,0.0,0.283,0.263799,0.0,0.0,0.0,1018.70624,0.0,1024.0,265.60138,10.440001
14031109,2024,4,12,1,7,19790,N944AT,3108.0,10397,30397,13,11267,31267,39,858,1030,92.0,432.0,B717,110.0,2.0,Jet,False,25.4,0,None,2024-12-01 08:58:00,5.665500,42.0,0.0,0.0,0.0,0.0,0.235,0.034101,-0.0,0.0,0.0,987.70013,0.0,1024.8,302.471160,11.520000,2024-12-01 08:58:00,-0.882500,44.0,0.0,0.0,0.0,0.0,0.259,0.062000,0.0,0.0,0.0,983.30884,3.0,1021.4,239.98172,39.600000
3762622,2023,3,7,13,4,19805,N968NN,744.0,11042,30647,39,11298,30194,48,1752,1942,170.0,1021.0,B738,160.0,2.0,Jet,False,10.0,0,None,2023-07-13 17:52:00,22.463001,78.0,0.0,0.0,0.0,0.0,0.261,0.314330,0.0,0.0,14.0,982.31160,0.0,1009.4,338.198520,18.359999,2023-07-13 17:52:00,40.026500,35.0,0.0,0.0,0.0,0.0,0.141,0.739295,0.0,0.0,0.0,986.32290,0.0,1006.5,177.70944,24.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8323095,2024,1,3,1,5,19805,N843NN,2336.0,11298,30194,48,11042,30647,39,1318,1646,148.0,1021.0,B738,172.0,2.0,Jet,False,14.7,0,None,2024-03-01 13:18:00,13.876500,68.0,0.0,0.0,0.0,0.0,0.333,0.330432,0.0,0.0,40.0,996.24630,1.0,1018.5,167.125000,21.960000,2024-03-01 13:18:00,9.912999,30.0,0.0,0.0,0.0,0.0,0.284,0.335583,0.0,95.0,0.0,996.77966,3.0,1025.5,154.72232,23.400000
8495898,2024,1,3,10,7,20397,N619NN,5156.0,10135,30135,42,11057,31057,37,1152,1348,116.0,481.0,CRJ9,88.0,2.0,Jet,False,7.7,0,None,2024-03-10 11:52:00,6.980500,69.0,0.0,0.0,0.0,0.0,0.314,0.128352,0.0,0.0,99.0,985.26030,3.0,999.4,258.503480,39.600000,2024-03-10 11:52:00,12.724000,46.0,0.0,0.0,0.0,0.0,0.290,0.349260,0.0,0.0,2.0,985.59235,0.0,1011.3,265.10098,38.160000
4704250,2023,3,8,

### Pipeline Functions

In [61]:
# Custom progress logger
class ProgressLogger(BaseEstimator, TransformerMixin):
    """
    A transformer that logs progress through a pipeline.
    """
    
    def __init__(self, total_rows, log_interval=0.01, name='Pipeline'):
        self.total_rows = total_rows
        self.log_interval = log_interval
        self.name = name
        self.start_time = None
        self.last_log_percent = -1
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Initialize timer if not started
        if self.start_time is None:
            self.start_time = time.time()
            print(f"{self.name} processing started on {self.total_rows:,} rows")
        
        # Calculate current progress
        current_rows = X.shape[0]
        percent_complete = current_rows / self.total_rows
        
        # Check if we need to log progress
        int_percent = int(percent_complete / self.log_interval)
        if int_percent > self.last_log_percent:
            self.last_log_percent = int_percent
            elapsed = time.time() - self.start_time
            
            # Estimate time remaining
            percent_done = percent_complete * 100
            if percent_complete > 0:
                total_est = elapsed / percent_complete
                remaining = total_est - elapsed
                time_str = f" - Est. remaining: {remaining:.1f}s"
            else:
                time_str = ""
                
            print(f"{self.name}: {percent_done:.1f}% complete ({current_rows:,}/{self.total_rows:,} rows){time_str}")
        
        return X


class DelayTrendEncoder(BaseEstimator, TransformerMixin):
    """
    Transformer that precomputes a route-based delay trend table using the training set,
    then merges it into any dataset based on (day_of_year, origin, dest).
    This allows for valid use in future (unlabeled) prediction data.

    Parameters:
        date_col (str): Name of the datetime column
        origin_col (str): Column name for origin airport ID
        dest_col (str): Column name for destination airport ID
        status_col (str): Target column used to derive delay signal (only during fit)
        output_col (str): Name of the output trend feature
    """

    def __init__(self,
                 date_col='dep_datetime',
                 origin_col='OriginAirportID',
                 dest_col='DestAirportID',
                 status_col='Flight_Status',
                 output_col='Delay_Trend'):
        self.date_col = date_col
        self.origin_col = origin_col
        self.dest_col = dest_col
        self.status_col = status_col
        self.output_col = output_col

    def fit(self, X, y=None):
    # Create a copy to avoid modifying the original
        X_temp = X.copy()
        X_temp[self.date_col] = pd.to_datetime(X_temp[self.date_col])
        X_temp['day_of_year'] = X_temp[self.date_col].dt.dayofyear
        
        # Add the target temporarily for calculation only
        X_temp['Flight_Status'] = y.reset_index(drop=True)

        delay_reasons = [
            'CarrierDelay', 'WeatherDelay', 'NASDelay',
            'SecurityDelay', 'LateAircraftDelay'
        ]

        # Vectorized approach for delay signal calculation
        X_temp['delay_signal'] = 0
        
        # Create mask for valid delay statuses
        delay_mask = X_temp['Flight_Status'].str.contains('Delay', na=False)
        delay_types = X_temp.loc[delay_mask, 'Flight_Status'].str.split(' - ').str[-1]
        valid_delays = delay_types.isin(delay_reasons)
        
        # Get indices where both conditions are met and set delay_signal
        X_temp.loc[delay_mask[delay_mask].index[valid_delays], 'delay_signal'] = 1

        # Create trend table
        trend_table = (
            X_temp.groupby(['day_of_year', self.origin_col, self.dest_col])['delay_signal']
            .mean()
            .reset_index()
            .rename(columns={'delay_signal': self.output_col})
        )

        self.trend_lookup_ = trend_table
        
        # Don't modify the original X
        return self

    def transform(self, X):
        X = X.copy()
        X[self.date_col] = pd.to_datetime(X[self.date_col])
        X['day_of_year'] = X[self.date_col].dt.dayofyear

        X = X.merge(
            self.trend_lookup_,
            on=['day_of_year', self.origin_col, self.dest_col],
            how='left'
        )

        X[self.output_col] = X[self.output_col].fillna(0)
        return X.drop(columns=['day_of_year'])

class SameDayTailReuseEncoder(BaseEstimator, TransformerMixin):
    """
    Transformer that creates a feature counting how many times
    the same tail number (aircraft) is used on the same day.

    Parameters:
        datetime_col (str): Column containing full departure datetime
        tail_col (str): Column name for tail number
        output_col (str): Name of the output column
    """

    def __init__(self,
                 datetime_col='dep_datetime',
                 tail_col='Tail_Number',
                 output_col='Same_Day_Tail_Reuse'):
        self.datetime_col = datetime_col
        self.tail_col = tail_col
        self.output_col = output_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Ensure datetime is parsed correctly
        X[self.datetime_col] = pd.to_datetime(X[self.datetime_col])

        # Extract just the date portion
        X['dep_date'] = X[self.datetime_col].dt.date

        # Count reuse of same tail number per day
        X[self.output_col] = (
            X.groupby([self.tail_col, 'dep_date'])[self.tail_col]
            .transform('count')
            .astype(float)  
        )

        return X.drop(columns=['dep_date'])

    
class TurnaroundDelayEncoder(BaseEstimator, TransformerMixin):
    """
    Transformer that computes:
    - Previous_Flight_Delay: scheduled departure time of previous flight with same tail number on same day
    - Turnaround_Time: time between previous arrival and current departure

    Parameters:
        datetime_col (str): Column with full departure datetime
        dep_time_col (str): Column with scheduled departure time (e.g. CRSDepTime)
        arr_time_col (str): Column with scheduled arrival time (e.g. CRSArrTime)
        tail_col (str): Tail number column
        output_prefix (str): Prefix to use for new feature columns
    """

    def __init__(self,
                 datetime_col='dep_datetime',
                 dep_time_col='CRSDepTime',
                 arr_time_col='CRSArrTime',
                 tail_col='Tail_Number',
                 output_prefix=''):
        self.datetime_col = datetime_col
        self.dep_time_col = dep_time_col
        self.arr_time_col = arr_time_col
        self.tail_col = tail_col
        self.output_prefix = output_prefix

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.datetime_col] = pd.to_datetime(X[self.datetime_col])
        X['dep_date'] = X[self.datetime_col].dt.date

        # Sort the values
        X = X.sort_values(by=[self.tail_col, 'dep_date', self.dep_time_col])

        prev_delay_col = self.output_prefix + 'Previous_Flight_Delay'
        turnaround_col = self.output_prefix + 'Turnaround_Time'

        # Previous scheduled departure
        X[prev_delay_col] = (
            X.groupby([self.tail_col, 'dep_date'])[self.dep_time_col]
            .shift(1)
        )

        # Previous scheduled arrival time
        X['Previous_Arrival_Time'] = (
            X.groupby([self.tail_col, 'dep_date'])[self.arr_time_col]
            .shift(1)
        )

        # Compute turnaround time using a defined method
        X[turnaround_col] = self._calculate_turnaround(X[self.dep_time_col], X['Previous_Arrival_Time'])

        # Replace missing with 0
        X[prev_delay_col] = X[prev_delay_col].fillna(0)
        X[turnaround_col] = X[turnaround_col].fillna(0)

        return X.drop(columns=['dep_date', 'Previous_Arrival_Time'])

    def _calculate_turnaround(self, current_dep, previous_arr):
        """
        Applies time difference logic with wrap-around at midnight (2400).
        """
        diff = current_dep - previous_arr
        adjusted = diff.mask((~diff.isna()) & (diff < 0), diff + 2400)
        return adjusted

class SlackTimeEncoder(BaseEstimator, TransformerMixin):
    """
    Transformer that calculates slack time as the difference between scheduled
    arrival time (CRSArrTime) and scheduled elapsed time (CRSElapsedTime).

    Parameters:
        arr_col (str): Column name for scheduled arrival time.
        elapsed_col (str): Column name for scheduled elapsed time.
        output_col (str): Name of the output column to store slack time.
    """

    def __init__(self,
                 arr_col='CRSArrTime',
                 elapsed_col='CRSElapsedTime',
                 output_col='Slack_Time'):
        self.arr_col = arr_col
        self.elapsed_col = elapsed_col
        self.output_col = output_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.output_col] = X[self.arr_col] - X[self.elapsed_col]
        return X

In [62]:
# Apply the fitted DelayTrendEncoder to the data
encoder = DelayTrendEncoder(
    date_col='dep_datetime',
    origin_col='OriginAirportID',
    dest_col='DestAirportID',
    status_col='Flight_Status',
    output_col='Delay_Trend'
)

# Fit on data using target label
encoder.fit(flight_data, flight_data['Flight_Status'])

# Transform the data
data_transformed = encoder.transform(flight_data)

# Create dummy variables for Flight_Status categories
status_dummies = pd.get_dummies(flight_data['Flight_Status'], prefix='Status')

# Add trend feature to the dummy DataFrame
correlation_df = pd.concat([status_dummies, data_transformed[['Delay_Trend']]], axis=1)

# Calculate correlation matrix
correlation_matrix = correlation_df.corr()

# Plot heatmap for Delay_Trend against all status categories
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix[['Delay_Trend']], annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Delay_Trend vs Flight_Status Categories')
plt.show()


KeyboardInterrupt: 

### Pipeline Model

In [63]:
# import random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


numerical_features = [
    'Distance', 'relative_humidity_2m', 'temperature_2m',
    'Delay_Trend', 'Same_Day_Tail_Reuse', 'Previous_Flight_Delay',
    'Turnaround_Time', 'Slack_Time',
    'Aircraft_Seats', 'Aircraft_Engines', 'Aircraft_Age', 'Is_Freighter',
    'Is_Holiday_Week',
    'temperature_2m_dest_dep_time', 'relative_humidity_2m_dest_dep_time',
    'precipitation', 'showers', 'snowfall', 'snow_depth',
    'soil_moisture_0_to_1cm', 'et0_fao_evapotranspiration', 'evapotranspiration',
    'cloud_cover_high', 'cloud_cover_low', 'surface_pressure', 'weather_code',
    'pressure_msl', 'wind_direction_10m', 'wind_gusts_10m',
    'precipitation_dest_dep_time', 'showers_dest_dep_time',
    'snowfall_dest_dep_time', 'snow_depth_dest_dep_time',
    'soil_moisture_0_to_1cm_dest_dep_time',
    'et0_fao_evapotranspiration_dest_dep_time',
    'evapotranspiration_dest_dep_time', 'cloud_cover_high_dest_dep_time',
    'cloud_cover_low_dest_dep_time', 'surface_pressure_dest_dep_time',
    'weather_code_dest_dep_time', 'pressure_msl_dest_dep_time',
    'wind_direction_10m_dest_dep_time', 'wind_gusts_10m_dest_dep_time'
]



# Define your numeric transformer
numeric_transformer = Pipeline(steps=[
    ('log1p_distance', ColumnTransformer([
        ('log', FunctionTransformer(np.log1p), ['Distance'])
    ], remainder='passthrough', verbose_feature_names_out=False)),
    
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Full pipeline with progress loggers
preprocessor = Pipeline([
    ('initial_logger', ProgressLogger(total_rows=13000000, name='Starting Pipeline')),
    
    ('delay_trend', DelayTrendEncoder(
        date_col='dep_datetime',
        origin_col='OriginAirportID',
        dest_col='DestAirportID',
        status_col='Flight_Status',
        output_col='Delay_Trend'
    )),
    
    ('trend_logger', ProgressLogger(total_rows=13000000, name='Delay Trend Complete')),
    
    ('tail_reuse', SameDayTailReuseEncoder(
        datetime_col='dep_datetime',
        tail_col='Tail_Number',
        output_col='Same_Day_Tail_Reuse'
    )),
    
    ('reuse_logger', ProgressLogger(total_rows=13000000, name='Tail Reuse Complete')),
    
    ('turnaround_delay', TurnaroundDelayEncoder(
        datetime_col='dep_datetime',
        dep_time_col='CRSDepTime',
        arr_time_col='CRSArrTime',
        tail_col='Tail_Number',
        output_prefix=''
    )),
    
    ('turnaround_logger', ProgressLogger(total_rows=13000000, name='Turnaround Delay Complete')),
    
    ('slack_time', SlackTimeEncoder(
        arr_col='CRSArrTime',
        elapsed_col='CRSElapsedTime',
        output_col='Slack_Time'
    )),
    
    ('slack_logger', ProgressLogger(total_rows=13000000, name='Slack Time Complete')),
    
    ('feature_prep', ColumnTransformer([
        ('numeric', numeric_transformer, numerical_features),
        ('drop', 'drop', [
            'Aircraft_Model', 'Aircraft_EngineType', 'Holiday',
            'dest_dep_datetime', 'dep_datetime', 'Tail_Number'
        ])
    ], remainder='passthrough', verbose_feature_names_out=False)),
    
    ('prep_logger', ProgressLogger(total_rows=13000000, name='Feature Prep Complete')),
    
    ('select', SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))),
    
    ('final_logger', ProgressLogger(total_rows=13000000, name='Pipeline Complete'))
])


preprocessor

Pipeline(steps=[('initial_logger',
                 ProgressLogger(name='Starting Pipeline', total_rows=13000000)),
                ('delay_trend', DelayTrendEncoder()),
                ('trend_logger',
                 ProgressLogger(name='Delay Trend Complete',
                                total_rows=13000000)),
                ('tail_reuse', SameDayTailReuseEncoder()),
                ('reuse_logger',
                 ProgressLogger(name='Tail Reuse Complete',
                                total_rows=13000000)),
                ('turnaround_delay', T...
                                                   'Aircraft_EngineType',
                                                   'Holiday',
                                                   'dest_dep_datetime',
                                                   'dep_datetime',
                                                   'Tail_Number'])],
                                   verbose_feature_names_out=False)),
                ('prep_logger',
                 ProgressLogger(name='Feature Prep Complete',
                                total_rows=13000000)),
                ('select',
                 SelectFromModel(estimator=RandomForestClassifier(n_jobs=-1,
                                                                  random_state=42))),
                ('final_logger',
                 ProgressLogger(name='Pipeline Complete',
                                total_rows=13000000))])

In [52]:
X

,Year,Quarter,Month,DayofMonth,DayOfWeek,DOT_ID_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginCityMarketID,OriginStateFips,DestAirportID,DestCityMarketID,DestStateFips,CRSDepTime,CRSArrTime,CRSElapsedTime,Distance,Aircraft_Model,Aircraft_Seats,Aircraft_Engines,Aircraft_EngineType,Is_Freighter,Aircraft_Age,Is_Holiday_Week,Holiday,dep_datetime,temperature_2m,relative_humidity_2m,precipitation,showers,snowfall,snow_depth,soil_moisture_0_to_1cm,et0_fao_evapotranspiration,evapotranspiration,cloud_cover_high,cloud_cover_low,surface_pressure,weather_code,pressure_msl,wind_direction_10m,wind_gusts_10m,dest_dep_datetime,temperature_2m_dest_dep_time,relative_humidity_2m_dest_dep_time,precipitation_dest_dep_time,showers_dest_dep_time,snowfall_dest_dep_time,snow_depth_dest_dep_time,soil_moisture_0_to_1cm_dest_dep_time,et0_fao_evapotranspiration_dest_dep_time,evapotranspiration_dest_dep_time,cloud_cover_high_dest_dep_time,cloud_cover_low_dest_dep_time,surface_pressure_dest_dep_time,weather_code_dest_dep_time,pressure_msl_dest_dep_time,wind_direction_10m_dest_dep_time,wind_gusts_10m_dest_dep_time
5556962,2023,4,10,9,1,19790,N336NW,2101.0,13487,31650,27,11637,31637,38,2255,2356,61.0,223.0,A320,NaN,2.0,Jet,False,32.6,1,Columbus Day,2023-10-09 22:55:00,6.893500,57.0,0.0,0.0,0.0,0.0,0.089,0.006185,0.0,0.0,0.0,980.69684,0.0,1011.1,291.801480,9.000000,2023-10-09 22:55:00,5.256500,70.0,0.0,0.0,0.0,0.0,0.322,0.006884,-0.0,0.0,0.0,979.65765,0.0,1012.8,333.43503,15.480000
14416945,2024,4,12,20,5,19805,N156AN,2214.0,11298,30194,48,14262,34262,6,1230,1347,197.0,1126.0,A321,181.0,2.0,Jet,False,9.1,1,Christmas Day,2024-12-20 12:30:00,12.976501,37.0,0.0,0.0,0.0,0.0,0.224,0.076416,0.0,1.0,0.0,999.01373,0.0,1021.4,206.564990,15.840000,2024-12-20 12:30:00,23.974000,10.0,0.0,0.0,0.0,0.0,0.031,0.191728,0.0,55.0,0.0,1001.62115,2.0,1016.8,150.25520,3.960000
433147,2023,1,1,24,2,19805,N129AA,696.0,11298,30194,48,12892,32575,6,1805,1929,204.0,1235.0,A321,181.0,2.0,Jet,False,10.2,0,None,2023-01-24 18:05:00,4.526500,88.0,0.0,0.0,0.0,0.0,0.461,0.030539,0.0,0.0,100.0,987.39460,3.0,1010.2,39.289394,30.599998,2023-01-24 18:05:00,16.711000,45.0,0.0,0.0,0.0,0.0,0.283,0.263799,0.0,0.0,0.0,1018.70624,0.0,1024.0,265.60138,10.440001
14031109,2024,4,12,1,7,19790,N944AT,3108.0,10397,30397,13,11267,31267,39,858,1030,92.0,432.0,B717,110.0,2.0,Jet,False,25.4,0,None,2024-12-01 08:58:00,5.665500,42.0,0.0,0.0,0.0,0.0,0.235,0.034101,-0.0,0.0,0.0,987.70013,0.0,1024.8,302.471160,11.520000,2024-12-01 08:58:00,-0.882500,44.0,0.0,0.0,0.0,0.0,0.259,0.062000,0.0,0.0,0.0,983.30884,3.0,1021.4,239.98172,39.600000
3762622,2023,3,7,13,4,19805,N968NN,744.0,11042,30647,39,11298,30194,48,1752,1942,170.0,1021.0,B738,160.0,2.0,Jet,False,10.0,0,None,2023-07-13 17:52:00,22.463001,78.0,0.0,0.0,0.0,0.0,0.261,0.314330,0.0,0.0,14.0,982.31160,0.0,1009.4,338.198520,18.359999,2023-07-13 17:52:00,40.026500,35.0,0.0,0.0,0.0,0.0,0.141,0.739295,0.0,0.0,0.0,986.32290,0.0,1006.5,177.70944,24.480000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8323095,2024,1,3,1,5,19805,N843NN,2336.0,11298,30194,48,11042,30647,39,1318,1646,148.0,1021.0,B738,172.0,2.0,Jet,False,14.7,0,None,2024-03-01 13:18:00,13.876500,68.0,0.0,0.0,0.0,0.0,0.333,0.330432,0.0,0.0,40.0,996.24630,1.0,1018.5,167.125000,21.960000,2024-03-01 13:18:00,9.912999,30.0,0.0,0.0,0.0,0.0,0.284,0.335583,0.0,95.0,0.0,996.77966,3.0,1025.5,154.72232,23.400000
8495898,2024,1,3,10,7,20397,N619NN,5156.0,10135,30135,42,11057,31057,37,1152,1348,116.0,481.0,CRJ9,88.0,2.0,Jet,False,7.7,0,None,2024-03-10 11:52:00,6.980500,69.0,0.0,0.0,0.0,0.0,0.314,0.128352,0.0,0.0,99.0,985.26030,3.0,999.4,258.503480,39.600000,2024-03-10 11:52:00,12.724000,46.0,0.0,0.0,0.0,0.0,0.290,0.349260,0.0,0.0,2.0,985.59235,0.0,1011.3,265.10098,38.160000
4704250,2023,3,8,

In [64]:
# Train the pipeline in this cell and run it on X, this cell must end with the transformed data
X_prepped = preprocessor.fit_transform(X, y)
# X_prepped = preprocessor.fit_transform(X, y)
pd.set_option('display.max_columns', None)
X_prepped

Starting Pipeline processing started on 13,000,000 rows
Starting Pipeline: 7.7% complete (1,000,000/13,000,000 rows) - Est. remaining: 0.0s
Delay Trend Complete processing started on 13,000,000 rows
Delay Trend Complete: 7.7% complete (1,000,000/13,000,000 rows) - Est. remaining: 0.0s
Tail Reuse Complete processing started on 13,000,000 rows
Tail Reuse Complete: 7.7% complete (1,000,000/13,000,000 rows) - Est. remaining: 0.0s
Turnaround Delay Complete processing started on 13,000,000 rows
Turnaround Delay Complete: 7.7% complete (1,000,000/13,000,000 rows) - Est. remaining: 0.0s
Slack Time Complete processing started on 13,000,000 rows
Slack Time Complete: 7.7% complete (1,000,000/13,000,000 rows) - Est. remaining: 0.0s
Feature Prep Complete processing started on 13,000,000 rows
Feature Prep Complete: 7.7% complete (1,000,000/13,000,000 rows) - Est. remaining: 0.0s
Pipeline Complete processing started on 13,000,000 rows
Pipeline Complete: 7.7% complete (1,000,000/13,000,000 rows) - Est

,Distance,relative_humidity_2m,temperature_2m,Slack_Time,Aircraft_Seats,Aircraft_Age,temperature_2m_dest_dep_time,relative_humidity_2m_dest_dep_time,soil_moisture_0_to_1cm,et0_fao_evapotranspiration,surface_pressure,pressure_msl,wind_direction_10m,wind_gusts_10m,soil_moisture_0_to_1cm_dest_dep_time,et0_fao_evapotranspiration_dest_dep_time,surface_pressure_dest_dep_time,pressure_msl_dest_dep_time,wind_direction_10m_dest_dep_time,wind_gusts_10m_dest_dep_time,DayofMonth,Flight_Number_Operating_Airline,OriginAirportID,OriginCityMarketID,DestAirportID,DestCityMarketID,CRSDepTime,CRSArrTime,CRSElapsedTime
906222,0.369357,1.203517,-1.445007,-0.514526,0.0,1.327388e-16,-0.174294,-0.262195,0.924893,-0.633984,-0.010703,0.363410,0.197994,1.634310,-1.663581,0.199781,0.764083,1.012172,-1.309257,-0.500461,1,413.0,11193,33105,14986,34986,1000,1225,145.0
46616,0.317797,-0.353390,-0.839192,-0.117544,0.0,1.327388e-16,0.772821,-0.573192,0.875459,0.410852,0.791616,1.210635,-0.686829,-0.219877,-1.811813,1.131100,0.787925,1.089191,-0.955471,-0.287724,15,516.0,13931,33667,11697,32467,1207,1429,142.0
581367,0.657118,1.025585,-0.142452,-0.725483,0.0,1.327388e-16,-1.198505,1.026222,-0.973376,-0.334837,0.538843,-0.437602,-0.004209,2.303034,0.866253,-0.626410,-0.135158,-2.438282,0.287024,0.715175,23,1242.0,14761,34761,14576,34576,850,1126,156.0
233881,0.498280,-0.086491,-0.914273,1.623809,0.0,1.327388e-16,0.009060,0.048803,-0.923942,-0.806326,-0.089145,-1.423463,-0.671644,0.357657,-1.554877,-0.713539,0.719479,0.858134,-0.416605,0.502439,26,2877.0,15295,35165,14761,34761,2115,2340,145.0
353296,1.361632,0.714204,-2.595482,-0.092612,0.0,1.327388e-16,0.905632,0.404228,0.311911,-0.672912,-0.544256,1.580333,0.347203,0.357657,-0.793950,1.055978,0.632609,0.226578,-0.807499,-0.287724,8,1214.0,13433,33304,14761,34761,1040,1540,240.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808118,1.442489,0.491788,-0.988075,0.655325,0.0,1.327388e-16,-0.136165,-0.217766,0.964441,-0.587815,0.790997,1.226039,0.632900,-1.222962,-0.764304,-0.593125,0.720387,1.089191,-0.678236,-0.652415,29,2170.0,14057,34057,12266,31453,1330,1940,250.0
487037,0.276457,0.091441,-0.111209,0.124098,0.0,1.327388e-16,-1.772952,-0.573192,-1.101904,-0.591173,0.657288,0.763916,-1.786739,0.327260,0.658728,-0.712925,0.087333,1.150806,0.404056,1.322993,29,1749.0,13204,31454,11193,33105,1333,1555,142.0
113919,1.904001,-0.575805,-0.179894,-1.853143,0.0,1.327388e-16,0.506165,0.626369,0.489873,-0.067347,0.695314,0.425027,-1.100975,-0.918997,-1.732756,-0.709641,0.673529,0.288193,1.426285,-0.044597,29,1369.0,14771,32457,13303,32467,2242,714,332.0
102413,0.211924,-0.486839,-0.165773,1.111759,0.0,1.327388e-16,-1.759159,-0.795333,-1.408396,-0.487208,0.746487,0.810129,-1.801584,0.205674,0.520378,-0.753398,0.129976,1.227825,0.394611,0.927911,30,977.0,14112,33195,12223,32223,1848,2045,117.0


In [ ]:
# # pickle the preprocessor
# import pickle
# with open("preprocessor.pkl", "wb") as f:
#     pickle.dump(preprocessor, f, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
# use MLP classifier on the pipeline

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, precision_score, f1_score

X_test = test_data.drop('Flight_Status', axis=1)
y_test = test_data['Flight_Status']

# Your X_prepped is already available from previous preprocessing
# Now we'll create and train the MLP classifier optimized for precision

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, precision_score

# Create the MLP classifier with precision-focused parameters
mlp = MLPClassifier(
    hidden_layer_sizes=(100, 50),  # Two hidden layers
    activation='relu',             
    solver='adam',                 
    alpha=0.001,                   # Higher regularization for better precision
    batch_size='auto',             
    learning_rate='adaptive',      
    max_iter=2000,                 # More iterations to ensure convergence
    early_stopping=True,           # Stop if validation performance doesn't improve
    validation_fraction=0.2,       # Portion of training data used for validation
    n_iter_no_change=20,           # Patience parameter
    random_state=42,
)

# Train the model using your preprocessed X data
mlp.fit(X_prepped, y)

# For testing, you'll need to apply the same preprocessing to X_test
# If you haven't already done this, you would typically do:
# X_test_prepped = preprocessor.transform(X_test)
# But since you didn't specify this step, I'll assume you need to do it:

# Make predictions (using whatever preprocessed test data you have)
# If X_test is already preprocessed:
y_pred = mlp.predict(X_test)
# If you need to preprocess X_test:
# y_pred = mlp.predict(X_test_prepped)

# Evaluate with focus on precision
print("Precision score:", precision_score(y_test, y_pred, average='macro'))
print("F1 score:", f1_score(y_test, y_pred, average='macro'))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

ValueError: Input X contains NaN.
MLPClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values